# BLAES Units Population Activity Analyses

This notebook contains code for running group-level analyses on population firing dynamics for the BLAES Units project.

---

> *Contact: Justin Campbell (justin.campbell@hsc.utah.edu)*  
> *Version: 1/15/2025*

## 1. Import Libraries & Configure Notebook

In [ ]:
import os
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# Display params
%matplotlib inline
# %config InlineBackend.figure_format='retina'
%config InlineBackend.figure_format = 'svg'

In [ ]:
# Palettes
print('Regions Palette:')
regionPal = ['#ff6e61', '#ffb84d', '#6d9dc5', '#5e4b8b']
sns.palplot(regionPal, .75)
plt.show()

print('Paired Palette:')
pairPal = ['#5e4b8b', '#ffb84d']
sns.palplot(pairPal, .75)
plt.show()

print('Alt. Paired Palette:')
altPairPal = ['#ff6e61', '#6d9dc5']
sns.palplot(altPairPal, .75)
plt.show()

In [ ]:
# Parameters
t = np.linspace(-4, 2, 200)
t_win = np.where((t >= -4) & (t <= 2), 1, 0)
t_stim = np.where((t >= 0) & (t <= 1), 1, 0)
trial_len = len(t)
fs = 30000
extractKDE = False
export = False

## 2. Extract Smooth FRs (KDEs) from Peri-Stimulation Rasters

In [ ]:
def get_KDE(epochs, condition, trial_info, unit, parse_outcomes = 'All'):
    '''
    Plot unit raster for peri-stimulation epochs.
    
    Arguments:
        epochs (np.ndarray): array of peri-stimulation epochs (indices)
        condition (str): condition label
        trial_info (pd.DataFrame): trial information
        unit (str): unit label
        parse_outcomes (str): outcome to parse by ('All', 'Hit', 'Miss')
        
    Returns:
        kde_vals_x (np.ndarray): x-values of KDE plot
        kde_vals_y (np.ndarray): y-values of KDE plot
    '''

    # Get unit data
    unitDF = events.copy()
    unitDF['Chan-Unit'] = unitDF['Channel'].astype(str) + '-' + unitDF['Unit'].astype(str)
    unitDF = unitDF[unitDF['Chan-Unit'] == unit]
    unitDF['TimeStamps'] = unitDF['TimeStamps'].astype('int')
    n_trials = epochs.shape[1]

    # Get epoch spikes
    epochSpikes = []
    for i in range(n_trials):
        epoch_start = epochs.iloc[:,i].min()
        stim_start = epoch_start + (4*fs)
        epoch_end = epochs.iloc[:,i].max()
        epochDF = unitDF[(unitDF['TimeStamps'] >= epoch_start) & (unitDF['TimeStamps'] <= epoch_end)]
        epochDF = epochDF.reset_index(drop = True)
        epochDF['TimeAdj'] = (epochDF['TimeStamps'] - stim_start) / fs
        epochDF['Trial'] = i + 1
        epochSpikes.append(epochDF)

    # Modify trial info to account for separated epochs during preprocessing
    trial_info['TrialAdj'] = 0
    stim_trial_counter = 1
    no_stim_trial_counter = 1
    for i in range(trial_info.shape[0]):
        if trial_info.iloc[i,1] == 1:
            trial_info.loc[i,'TrialAdj'] = stim_trial_counter
            stim_trial_counter += 1
        else:
            trial_info.loc[i,'TrialAdj'] = no_stim_trial_counter
            no_stim_trial_counter += 1
            
    # Parse by outcome
    if parse_outcomes == 'All':
        spikesDF = pd.concat(epochSpikes)
        spikesDF = spikesDF.reset_index(drop = True)
        
    elif parse_outcomes == 'Hit':
        spikesDF = pd.concat(epochSpikes)
        if condition == 'Stim':
            hits = trial_info[(trial_info['Outcome'] == 'Hit') & (trial_info['Condition'] == 1)]
            hit_trials = hits['TrialAdj'].values
            n_trials = hits.shape[0]
            spikesDF = spikesDF[spikesDF['Trial'].isin(hit_trials)]
        elif condition == 'No-Stim':
            hits = trial_info[(trial_info['Outcome'] == 'Hit') & (trial_info['Condition'] == 0)]
            hit_trials = hits['TrialAdj'].values
            n_trials = hits.shape[0]
            spikesDF = spikesDF[spikesDF['Trial'].isin(hit_trials)]
        spikesDF = spikesDF.reset_index(drop = True)
        
    elif parse_outcomes == 'Miss':
        spikesDF = pd.concat(epochSpikes)
        if condition == 'Stim':
            misses = trial_info[(trial_info['Outcome'] == 'Miss') & (trial_info['Condition'] == 1)]
            miss_trials = misses['TrialAdj'].values
            n_trials = misses.shape[0]
            spikesDF = spikesDF[spikesDF['Trial'].isin(miss_trials)]
        elif condition == 'No-Stim':
            misses = trial_info[(trial_info['Outcome'] == 'Miss') & (trial_info['Condition'] == 0)]
            miss_trials = misses['TrialAdj'].values
            n_trials = misses.shape[0]
            spikesDF = spikesDF[spikesDF['Trial'].isin(miss_trials)]
        spikesDF = spikesDF.reset_index(drop = True)
        
    # Calculate KDE values (smoothed FR)
    binSize = 0.1 # time (s)
    bins = np.arange(-4, 2.1, binSize)
    ps_spike_win = spikesDF.copy()
    ps_spike_win = ps_spike_win[ps_spike_win['TimeAdj'] >= -4]
    ps_spike_win = ps_spike_win[ps_spike_win['TimeAdj'] <= 2]
    ax = sns.histplot(ps_spike_win['TimeAdj'], bins = bins, kde = True, kde_kws = {'bw_adjust': 0.2, 'cut': 3, 'clip': [-4,2]}, color = 'k')
    
    # Extract KDE values
    try:
        kde_vals = ax.get_lines()[0].get_data()
        kde_vals_x = kde_vals[0]
        kde_vals_y = (kde_vals[1] * 10) / n_trials
    except:
        kde_vals_x = None
        kde_vals_y = None
        
    plt.close()
        
    return kde_vals_x, kde_vals_y

In [ ]:
# Define paths and load data
results_dir = '/Users/justincampbell/Library/CloudStorage/GoogleDrive-u0815766@gcloud.utah.edu/My Drive/Research Projects/BLAESUnits/Results'
save_dir = os.path.join(results_dir, 'Group', 'Figures')
stats_df = pd.read_csv(os.path.join(results_dir, 'Group/SpikeStats.csv'), index_col=0)
stats_df_valid = stats_df[stats_df['Valid'] == True]

In [ ]:
# Extract KDEs for Stim/No-Stim and Stim/No-Stim x Hit/Miss
if extractKDE:
    pID_list = []
    unit_list = []
    kde_y_stim_list = []
    kde_y_nostim_list = []
    kde_y_stim_hit_list = []
    kde_y_stim_miss_list = []
    kde_y_nostim_hit_list = []
    kde_y_nostim_miss_list = []

    for pID in stats_df_valid['pID'].unique():
        for unit in stats_df_valid[stats_df_valid['pID'] == pID]['Unit']:
            events = pd.read_csv(os.path.join(results_dir, pID, 'Events.csv'), index_col = 0)
            trial_info = pd.read_csv('/Users/justincampbell/Library/CloudStorage/GoogleDrive-u0815766@gcloud.utah.edu/My Drive/Research Projects/BLAESUnits/TrialInfo/' + pID + '_TrialInfo.csv')
            epochs_stim = pd.read_csv(os.path.join(results_dir, pID, 'StimEpochs.csv'), index_col = 0)
            epochs_nostim = pd.read_csv(os.path.join(results_dir, pID, 'NoStimEpochs.csv'), index_col = 0)
            
            x, y_stim = get_KDE(epochs_stim, 'Stim', trial_info, unit, 'All')
            _, y_nostim = get_KDE(epochs_nostim, 'No-Stim', trial_info, unit, 'All')
            _, y_stim_hit = get_KDE(epochs_stim, 'Stim', trial_info, unit, 'Hit')
            _, y_stim_miss = get_KDE(epochs_stim, 'Stim', trial_info, unit, 'Miss')
            _, y_nostim_hit = get_KDE(epochs_nostim, 'No-Stim', trial_info, unit, 'Hit')
            _, y_nostim_miss = get_KDE(epochs_nostim, 'No-Stim', trial_info, unit, 'Miss')
            
            pID_list.append(pID)
            unit_list.append(unit)
            kde_y_stim_list.append(y_stim)
            kde_y_nostim_list.append(y_nostim)
            kde_y_stim_hit_list.append(y_stim_hit)
            kde_y_stim_miss_list.append(y_stim_miss)
            kde_y_nostim_hit_list.append(y_nostim_hit)
            kde_y_nostim_miss_list.append(y_nostim_miss)
            
    # Set empty KDE values to NaN
    kde_y_stim_list = [[np.nan] * len(t)  if x is None else x for x in kde_y_stim_list]
    kde_y_nostim_list = [[np.nan] * len(t) if x is None else x for x in kde_y_nostim_list]
    kde_y_stim_hit_list = [[np.nan] * len(t) if x is None else x for x in kde_y_stim_hit_list]
    kde_y_stim_miss_list = [[np.nan] * len(t) if x is None else x for x in kde_y_stim_miss_list]
    kde_y_nostim_hit_list = [[np.nan] * len(t) if x is None else x for x in kde_y_nostim_hit_list]
    kde_y_nostim_miss_list = [[np.nan] * len(t) if x is None else x for x in kde_y_nostim_miss_list]
            
    # Save KDE values
    kde_df_stim = pd.DataFrame(kde_y_stim_list)
    kde_df_nostim = pd.DataFrame(kde_y_nostim_list)
    kde_df_stim_hit = pd.DataFrame(kde_y_stim_hit_list)
    kde_df_stim_miss = pd.DataFrame(kde_y_stim_miss_list)
    kde_df_nostim_hit = pd.DataFrame(kde_y_nostim_hit_list)
    kde_df_nostim_miss = pd.DataFrame(kde_y_nostim_miss_list)
    
    # Save unit info
    kde_df_info = pd.DataFrame({'pID': pID_list, 'Unit': unit_list})
    
    # # Add columns for Region, Valid, and Ipsilateral from df
    kde_df_info['Region'] = pd.merge(kde_df_info, stats_df_valid[['pID', 'Unit', 'Region']], on = ['pID', 'Unit'], how = 'left')['Region']
    kde_df_info['Valid'] = pd.merge(kde_df_info, stats_df_valid[['pID', 'Unit', 'Valid']], on = ['pID', 'Unit'], how = 'left')['Valid']
    kde_df_info['Ipsilateral'] = pd.merge(kde_df_info, stats_df_valid[['pID', 'Unit', 'Ipsilateral']], on = ['pID', 'Unit'], how = 'left')['Ipsilateral']
    
    # Export
    kde_df_info.to_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_Info.csv'))
    kde_df_stim.to_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_Stim.csv'))
    kde_df_nostim.to_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_NoStim.csv'))
    kde_df_stim_hit.to_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_Stim_Hit.csv'))
    kde_df_stim_miss.to_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_Stim_Miss.csv'))
    kde_df_nostim_hit.to_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_NoStim_Hit.csv'))
    kde_df_nostim_miss.to_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_NoStim_Miss.csv'))

## 3. Process FR Data

In [ ]:
# Load KDE data
kde_df_stim = pd.read_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_Stim.csv'), index_col = 0)
kde_df_nostim = pd.read_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_NoStim.csv'), index_col = 0)
kde_df_stim_hit = pd.read_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_Stim_Hit.csv'), index_col = 0)
kde_df_stim_miss = pd.read_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_Stim_Miss.csv'), index_col = 0)
kde_df_nostim_hit = pd.read_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_NoStim_Hit.csv'), index_col = 0)
kde_df_nostim_miss = pd.read_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_NoStim_Miss.csv'), index_col = 0)

# Get unit/trial info
kde_df_info = pd.read_csv(os.path.join(results_dir, 'Group/KDESmoothFRs_Info.csv'), index_col = 0)
regions = kde_df_info['Region']
laterality = kde_df_info['Ipsilateral']

In [ ]:
# Z-score kde_df for each unit
scaler = StandardScaler()
kde_stim_z = pd.DataFrame(scaler.fit_transform(kde_df_stim), columns = kde_df_stim.columns)
kde_nostim_z = pd.DataFrame(scaler.fit_transform(kde_df_nostim), columns = kde_df_nostim.columns)
kde_stim_hit_z = pd.DataFrame(scaler.fit_transform(kde_df_stim_hit), columns = kde_df_stim_hit.columns)
kde_stim_miss_z = pd.DataFrame(scaler.fit_transform(kde_df_stim_miss), columns = kde_df_stim_miss.columns)
kde_nostim_hit_z = pd.DataFrame(scaler.fit_transform(kde_df_nostim_hit), columns = kde_df_nostim_hit.columns)
kde_nostim_miss_z = pd.DataFrame(scaler.fit_transform(kde_df_nostim_miss), columns = kde_df_nostim_miss.columns)

## 4. PCA (Individual Components)

### 4.1 Run PCA

In [ ]:
# PCA parameters
n_components = 3
pca = PCA(n_components = n_components)

In [ ]:
# Parse data for PCA

# Condition
cond_df = pd.concat([kde_stim_z, kde_nostim_z], axis = 1)

# Regions (Stim)
hip_stim = kde_stim_z[regions == 'HIP']
ofc_stim = kde_stim_z[regions == 'OFC']
amy_stim = kde_stim_z[regions == 'AMY']
acc_stim = kde_stim_z[regions == 'ACC']

# Regions (No-Stim)
hip_nostim = kde_nostim_z[regions == 'HIP']
ofc_nostim = kde_nostim_z[regions == 'OFC']
amy_nostim = kde_nostim_z[regions == 'AMY']
acc_nostim = kde_nostim_z[regions == 'ACC']

# Region x Laterality (Stim)
hip_stim_ipsi = kde_stim_z[(regions == 'HIP') & (laterality == True)]
hip_stim_contra = kde_stim_z[(regions == 'HIP') & (laterality == False)]
ofc_stim_ipsi = kde_stim_z[(regions == 'OFC') & (laterality == True)]
ofc_stim_contra = kde_stim_z[(regions == 'OFC') & (laterality == False)]
amy_stim_ipsi = kde_stim_z[(regions == 'AMY') & (laterality == True)]
amy_stim_contra = kde_stim_z[(regions == 'AMY') & (laterality == False)]
acc_stim_ipsi = kde_stim_z[(regions == 'ACC') & (laterality == True)]
acc_stim_contra = kde_stim_z[(regions == 'ACC') & (laterality == False)]

# Remove NaNs (if they exist)
cond_df = cond_df.dropna(axis = 0)
hip_stim = hip_stim.dropna(axis = 0)
ofc_stim = ofc_stim.dropna(axis = 0)
amy_stim = amy_stim.dropna(axis = 0)
acc_stim = acc_stim.dropna(axis = 0)
hip_nostim = hip_nostim.dropna(axis = 0)
ofc_nostim = ofc_nostim.dropna(axis = 0)
amy_nostim = amy_nostim.dropna(axis = 0)
acc_nostim = acc_nostim.dropna(axis = 0)
hip_stim_ipsi = hip_stim_ipsi.dropna(axis = 0)
hip_stim_contra = hip_stim_contra.dropna(axis = 0)
ofc_stim_ipsi = ofc_stim_ipsi.dropna(axis = 0)
ofc_stim_contra = ofc_stim_contra.dropna(axis = 0)
amy_stim_ipsi = amy_stim_ipsi.dropna(axis = 0)
amy_stim_contra = amy_stim_contra.dropna(axis = 0)
acc_stim_ipsi = acc_stim_ipsi.dropna(axis = 0)
acc_stim_contra = acc_stim_contra.dropna(axis = 0)
stim_hit = kde_stim_hit_z.dropna(axis = 0)
stim_miss = kde_stim_miss_z.dropna(axis = 0)
nostim_hit = kde_nostim_hit_z.dropna(axis = 0)
nostim_miss = kde_nostim_miss_z.dropna(axis = 0)

In [ ]:
# Run PCA

# Condition
cond_pca = pca.fit_transform(cond_df.T) # combined in one matrix

# Regions x Condition
hip_stim_pca = pca.fit_transform(hip_stim.T)
ofc_stim_pca = pca.fit_transform(ofc_stim.T)
amy_stim_pca = pca.fit_transform(amy_stim.T)
acc_stim_pca = pca.fit_transform(acc_stim.T)
region_stim_pcas = [hip_stim_pca, ofc_stim_pca, amy_stim_pca, acc_stim_pca]
hip_nostim_pca = pca.fit_transform(hip_nostim.T)
ofc_nostim_pca = pca.fit_transform(ofc_nostim.T)
amy_nostim_pca = pca.fit_transform(amy_nostim.T)
acc_nostim_pca = pca.fit_transform(acc_nostim.T)
region_nostim_pcas = [hip_nostim_pca, ofc_nostim_pca, amy_nostim_pca, acc_nostim_pca]

# Region x Laterality (Stim)
hip_stim_ipsi_pca = pca.fit_transform(hip_stim_ipsi.T)
hip_stim_contra_pca = pca.fit_transform(hip_stim_contra.T)
ofc_stim_ipsi_pca = pca.fit_transform(ofc_stim_ipsi.T)
ofc_stim_contra_pca = pca.fit_transform(ofc_stim_contra.T)
amy_stim_ipsi_pca = pca.fit_transform(amy_stim_ipsi.T)
amy_stim_contra_pca = pca.fit_transform(amy_stim_contra.T)
acc_stim_ipsi_pca = pca.fit_transform(acc_stim_ipsi.T)
acc_stim_contra_pca = pca.fit_transform(acc_stim_contra.T)
region_stim_ipsi_pcas = [hip_stim_ipsi_pca, ofc_stim_ipsi_pca, amy_stim_ipsi_pca, acc_stim_ipsi_pca]
region_stim_contra_pcas = [hip_stim_contra_pca, ofc_stim_contra_pca, amy_stim_contra_pca, acc_stim_contra_pca]

# Outcome x Condition
hit_stim_pca = pca.fit_transform(stim_hit.T)
miss_stim_pca = pca.fit_transform(stim_miss.T)
hit_nostim_pca = pca.fit_transform(nostim_hit.T)
miss_nostim_pca = pca.fit_transform(nostim_miss.T)

### 4.2 Visualize PCA Components

In [ ]:
def plot_components(pca, labels, palette, export = False, save_str = None, title = None, legend = True):
    
    # Figure parameters
    fig, axes = plt.subplots(1, 3, figsize=[12, 2], sharey='row', sharex='row')
    
    if type(pca) == list:
        for i, ax in enumerate(axes):
            for ii, label in enumerate(labels):
                x = pca[ii][:, i]
                ax.plot(t, x, label=label, color = palette[ii], lw = 2.5)
    else:
        # Plot first 3 components
        for comp in range(3):
            ax = axes[comp]
            for i, label in enumerate(labels):
                x = pca[i * trial_len :(i+1) * trial_len, comp]
                ax.plot(t, x, label=labels[i], color=palette[i], lw = 2.5)
            
    # Aesthetics
    for i, ax in enumerate(axes):
        ax.axvspan(0, 1, color = 'grey', alpha = 0.125, zorder = -10)
        ax.axvline(-3, color = 'k', linestyle = '--', lw = 1, zorder = 10)
        ax.axvline(0, color = 'k', linestyle = '--', lw = 1, zorder = 10)
        ax.axvline(1, color = 'k', linestyle = '--', lw = 1, zorder = 10)
        ax.set_xlabel('Time (s)', fontsize = 'medium', labelpad = 10)
        ax.set_ylabel('PC{}'.format(i+1), fontsize = 'medium', labelpad = 5)
        ax.set_xlim([-4, 2])
        ax.set_yticks([-3, -1.5, 0, 1.5, 3], ['-3', '-1.5', '0', '1.5', '3'], fontsize = 'small')
        ax.set_xticks([-4, -3, -2, -1, 0, 1, 2], ['-4', '-3', '-2', '-1', '0', '1', '2'], fontsize = 'small')
        sns.despine(ax = ax)
        
        plt.legend(bbox_to_anchor=(1.025, 1), fontsize = 'small')
    if not legend:
        ax.legend().set_visible(False)
    
    if title:
        plt.suptitle(title, fontsize = 'large', y = 1.05)
    
    # Export
    if export:
        plt.savefig(os.path.join(save_dir, 'PCA_FR_' + save_str + '.pdf'), bbox_inches = 'tight')
    
    # Display
    plt.show()

In [ ]:
def plot_trajectories(pca, labels, palette, export = False, save_str = None):
    
    fig = plt.figure(figsize = (4,4))
    ax = fig.add_subplot(111, projection='3d')

    # PCA computed separately (multiple matrices)
    if type(pca) == list:
        for i, pca_x in enumerate(pca):
            prepost = pca_x[np.where(t_win == 1)[0]].copy()
            stim_on = pca_x[np.where(t_stim == 1)[0]].copy()
            
            x = prepost[:,0]
            y = prepost[:,1]
            z = prepost[:,2]
            
            x_stim = stim_on[:,0]
            y_stim = stim_on[:,1]
            z_stim = stim_on[:,2]
            
            ax.plot(x_stim, y_stim, z_stim, color = palette[i], lw = 2, zorder = 10)
            ax.plot(x, y, z, color = palette[i], ls = ':', lw = 1)
            ax.scatter(x[0], y[0], z[0], color = palette[i], s=25)
            ax.scatter(x[-1], y[-1], z[-1], color = palette[i], s=25, marker = 'x')
        
    # PCA computed together
    else:
        for i, x in enumerate(labels):
            group_pca = pca[i * trial_len :(i+1) * trial_len]

            prepost = group_pca[np.where(t_win == 1)[0]].copy()
            stim_on = group_pca[np.where(t_stim == 1)[0]].copy()

            x = prepost[:,0]
            y = prepost[:,1]
            z = prepost[:,2]

            x_stim = stim_on[:,0]
            y_stim = stim_on[:,1]
            z_stim = stim_on[:,2]

            ax.plot(x_stim, y_stim, z_stim, color = pairPal[i], lw = 2, zorder = 10)
            ax.plot(x, y, z, color = pairPal[i], ls = ':', lw = 1)
            ax.scatter(x[0], y[0], z[0], color = pairPal[i], s=25)
            ax.scatter(x[-1], y[-1], z[-1], color = pairPal[i], s=25, marker = 'x')

    # Aesthetics
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')
    ax.set_xlabel('PC 1')
    ax.set_ylabel('PC 2')
    ax.set_zlabel('PC 3')

    # specify the orientation of the 3d plot        
    ax.view_init(elev=15, azim=60)
    ax.set_box_aspect(None, zoom=0.85)
    plt.tight_layout()
    
    # Export
    if export:
        plt.savefig(os.path.join(save_dir, 'PCA_FR_3D_' + save_str + '.pdf'), bbox_inches = 'tight')
    
    # Display
    plt.show()

#### 4.2.1 Condition

In [ ]:
plot_components(cond_pca, ['Stim', 'No-Stim'], pairPal, export = export, save_str = 'Condition')
plot_trajectories(cond_pca, ['Stim', 'No-Stim'], pairPal, export = export, save_str = 'Condition')

#### 4.2.2 Region x Condition

In [ ]:
print('Stim')
plot_components(region_stim_pcas, ['HIP', 'OFC', 'AMY', 'ACC'], regionPal, export = export, save_str = 'StimRegion')
plot_trajectories(region_stim_pcas, ['HIP', 'OFC', 'AMY', 'ACC'], regionPal, export = export, save_str = 'StimRegion')

print('No-Stim')
plot_components(region_nostim_pcas, ['HIP', 'OFC', 'AMY', 'ACC'], regionPal, export = export, save_str = 'NoStimRegion')
plot_trajectories(region_nostim_pcas, ['HIP', 'OFC', 'AMY', 'ACC'], regionPal, export = export, save_str = 'NoStimRegion')

#### 4.2.3 Region x Laterality (Stim)

In [ ]:
print('Stim-Ipsilateral')
plot_components(region_stim_ipsi_pcas, ['HIP', 'OFC', 'AMY', 'ACC'], regionPal, export = export, save_str = 'StimIpsiRegion', title = 'Ipsilateral')
plot_trajectories(region_stim_ipsi_pcas, ['HIP', 'OFC', 'AMY', 'ACC'], regionPal, export = export, save_str = 'StimIpsiRegion')

print('Stim-Contralateral')
plot_components(region_stim_contra_pcas, ['HIP', 'OFC', 'AMY', 'ACC'], regionPal, export = export, save_str = 'StimContraRegion', title = 'Contralateral')
plot_trajectories(region_stim_contra_pcas, ['HIP', 'OFC', 'AMY', 'ACC'], regionPal, export = export, save_str = 'StimContraRegion')

#### 4.2.3 Outcome x Condition (Stim)

In [ ]:
print('NStim-Outcome')
plot_components([hit_stim_pca, miss_stim_pca], ['Hit', 'Miss'], altPairPal, export = export, save_str = 'StimOutcome')
plot_trajectories([hit_stim_pca, miss_stim_pca], ['Hit', 'Miss'], altPairPal, export = export, save_str = 'StimOutcome')

In [ ]:
print('No-Stim-Outcome')
plot_components([hit_nostim_pca, miss_nostim_pca], ['Hit', 'Miss'], altPairPal, export = export, save_str = 'NoStimOutcome')
plot_trajectories([hit_nostim_pca, miss_nostim_pca], ['Hit', 'Miss'], altPairPal, export = export, save_str = 'NoStimOutcome')